In [1]:
%pip install "transformers" "accelerate" "langchain" "einops"

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 8.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install langchain
%pip install sentence_transformers
%pip install chromadb==0.3.29

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# from google.colab import drive
# drive.mount("/content/drive",force_remount = True)


Mounted at /content/drive


In [50]:
import torch
from transformers import pipeline

generate_text = pipeline(model="lilloukas/GPlatty-30B", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)


In [31]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline

In [32]:
# create huggingface model pipeline
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)

In [33]:
# Enter path where vectorDBs are stored
%cd ~

/home/bhumik


In [34]:
# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [35]:
# Define ChromaDB parameters
persist_directory = "AllMini_Chroma_Tik_400" # path to specific vector database
CHROMA_SETTINGS = Settings(
        chroma_db_impl='duckdb+parquet',
        persist_directory=persist_directory,
        anonymized_telemetry=False
)

In [36]:
# load vectorDB instance
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS)

In [37]:
# Run sample query on vectorDB for context retrieval
query = "What are the placement statistics?"
docs = db.similarity_search_with_score(query)
context = docs[0][0].page_content
context

' on Tenure Faculty on Contract Need based Faculty on Contract Postdoc Positions Project Positions Staff Positions Placements Breadcrumb Home Placements About Placements The Placement Cell at DA-IICT is a concept in its own form. It has the following structure consisting of three important parts viz. the placement office consisting of the Manager (Placements) and placement secretary, the faculty members and finally the student representatives from each program. The structure aims at touching all the aspects required for the smooth functioning of the placement process. Campus placements at DA-IICT provide the students with a foot-in- the-door opportunity, enabling them to start off their career right after they have completed their course curriculum. Furthermore, they get to interact and engage with the industry professionals during the placement drives, which further help lay a foundation for their prospective career in the future as they familiarise with potential contacts from their 

In [46]:
from langchain import PromptTemplate, LLMChain 
from langchain.chains import RetrievalQA

# Custom prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. Only answer things existing in the database, don't try to make up an answer.
{context}
Question: {question} Explain in points.
Helpful Answer:"""

# langchain prompt template
prompt_with_context = PromptTemplate(
input_variables=["question", "context"],
template= prompt_template)

# Prompt template argument to be passed in RetrievalQA chain
chain_type_kwargs = {"prompt": prompt_with_context}

In [47]:
from langchain.memory import ConversationBufferMemory
# load when storing previous chat history
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [48]:
# Using langchain RetrievalQA chain for answer generation
qa = RetrievalQA.from_chain_type(llm=hf_pipeline, chain_type="stuff", retriever=db.as_retriever(),chain_type_kwargs=chain_type_kwargs, verbose=True)

In [49]:
import time
# Run cli interface of BOT
while True:
        query = input("\nEnter a query: ")
        if query == "exit":
            break
        if query.strip() == "":
            continue

        # Get the answer from the chain
        start = time.time()
        res = qa(query)
        answer, docs = res['result'], []
        end = time.time()

        # Print the result
        print("\n\n> Question:")
        print(query)
        print(f"\n> Answer (took {round(end - start, 2)} s.):")
        print(answer)



> Entering new RetrievalQA chain...

> Finished chain.


> Question:
Name any 3 computer science professors

> Answer (took 15.69 s.):

1. Prof. Anand Mishra
2. Prof. Bhaskar Chaudhury
3. Prof. Anurag Khanna


> Entering new RetrievalQA chain...

> Finished chain.


> Question:
Give placement statistics

> Answer (took 66.4 s.):

The placement statistics have been increasing steadily, thanks to the dedication of the students and the placement cell. The students are getting a foot-in- the- door opportunity after they have completed their course curriculum. It was seen that students are now familiar with the group discussion etiquette and are better able to put their ideas across the table. A thorough process has been put in place for preparing the students for the actual placements. The issues encountered include the need for more tests and more frequency for conducting the Group Discussions. A resource required to implement the practice is identified. We have thanked our alumnae for 